# Determination of lambda

The purpose of this notebook is to settle on a parameter lambda for the objective function.

In [1]:
# Adapt path to allow imports from project
import os
os.chdir("..")

In [2]:
# Relevant imports from project
from src.env.objectives import SingleLinear
from src.baseline.evaluate import best_single_treatment
from src.env.cell_lines import retrieve_lines
from src.env.drugs import DRUGS
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
# need function that identifies best single cell treatment for given objective
# iterate through candidates for lambda
# create plot
# look at single drug plots
# look at combined drug plots
# look at single and multi cell plots

### Create plots using single treatment baseline

In [4]:
# proliferation
def plot_tradeoff(line, lambdas, drugs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    fig.suptitle('Penalty tradeoff: ' + line)
    
    
    for d in drugs:
        res = [best_single_treatment([line], lambd=l, drugs=[d], max_dosage=8000, path="./../artifacts/baselines/") for l in lambdas]
        ds = [r[1] for r in res]
        ys = [r[2] for r in res]
        l1, = ax1.plot(lambdas, ys)
        l1.set_label(d)
        l2, = ax2.plot(lambdas, ds)
        l2.set_label(d)
        
    ax1.set_xscale('log')
    ax1.set_xlabel("lambda")
    ax1.set_ylabel("relative proliferation")
    
    ax2.set_xscale('log')
    ax2.set_xlabel("lambda")
    ax2.set_ylabel("dosage (nm)")
    ax2.legend()

    plt.show()
    
# proliferation
def plot_average(lines, name, lambdas): # plot single best with light coloring

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    fig.suptitle('Average: ' + name + " tissue cell-lines")
    
    res = {}
    ds = [0 for _ in lambdas]
    ys = [0 for _ in lambdas]
    for line in lines:
        print(line)
        res[line] = [best_single_treatment([line], lambd=l, max_dosage=8000, path="./../artifacts/baselines/") for l in lambdas]
        dd = [r[1] for r in res[line]]
        yy = [r[2] for r in res[line]]
        ax1.plot(lambdas, yy, alpha=0.1, color="gray")
        ax2.plot(lambdas, dd, alpha=0.1, color="gray")
        
        for i, _ in enumerate(lambdas):
            ds[i] += res[line][i][1]
            ys[i] += res[line][i][2]

    ds = [d / len(lines) for d in ds]
    ys = [y / len(lines) for y in ys]

    ax1.plot(lambdas, ys, color="blue", alpha=1.0)
    ax1.set_xscale('log')
    ax1.set_xlabel("lambda")
    ax1.set_ylabel("relative proliferation")

    ax2.plot(lambdas, ds, color="blue")
    ax2.set_xscale('log')
    ax2.set_xlabel("lambda")
    ax2.set_ylabel("dosage (nm)")

    plt.savefig("./notebooks/plots/average_" + name +  ".png")
    plt.show()
    
def plot_worst(lines, name, lambdas):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    fig.suptitle('Worst case: ' + name + " tissue cell-lines")
    
    res = {}
    ds = [0 for _ in lambdas]
    ys = [0 for _ in lambdas]
    for line in lines:
        print(line)
        res[line] = [best_single_treatment([line], lambd=l, max_dosage=8000, path="./../artifacts/baselines/") for l in lambdas]
        dd = [r[1] for r in res[line]]
        yy = [r[2] for r in res[line]]
        ax1.plot(lambdas, yy, alpha=0.1, color="gray")
        ax2.plot(lambdas, dd, alpha=0.1, color="gray")
        
        for i, _ in enumerate(lambdas):
            if ys[i] <= res[line][i][2]:
                ds[i] = res[line][i][1]
            ys[i] = max(ys[i], res[line][i][2])

    ds = [d for d in ds]
    ys = [y for y in ys]

    ax1.plot(lambdas, ys, color="blue", alpha=1.0)
    ax1.set_xscale('log')
    ax1.set_xlabel("lambda")
    ax1.set_ylabel("relative proliferation")

    ax2.plot(lambdas, ds, color="blue")
    ax2.set_xscale('log')
    ax2.set_xlabel("lambda")
    ax2.set_ylabel("dosage (nm)")

    plt.savefig("./notebooks/plots/max_" + name +  ".png")
    plt.show()

In [5]:
lambdas = [10 ** (-0.1 * i) for i in range(100)]

In [ ]:
plot_average(retrieve_lines("skin"), "skin", lambdas)

In [ ]:
plot_average(retrieve_lines("intestine"), "large-intestine", lambdas)

In [ ]:
plot_worst(retrieve_lines("skin"), "skin", lambdas)

In [ ]:
plot_worst(retrieve_lines("intestine"), "large-intestine", lambdas)

In [6]:
# created pdf with overview plots
cell_lines = []
cell_lines = retrieve_lines("skin")
cell_lines.extend(retrieve_lines("intestine"))

In [7]:
with PdfPages('./notebooks/plots/individual_lambda.pdf') as pdf:
    
    for line in cell_lines:
        print(line)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
        fig.suptitle('Penalty tradeoff: ' + line)

        for d in DRUGS:       
            res = [best_single_treatment([line], lambd=l, drugs=[d], max_dosage=8000, path="./../artifacts/baselines/") for l in lambdas]
            ds = [r[1] for r in res]
            ys = [r[2] for r in res]
            l1, = ax1.plot(lambdas, ys)
            l1.set_label(d)
            l2, = ax2.plot(lambdas, ds)
            l2.set_label(d)

        ax1.set_xscale('log')
        ax1.set_xlabel("lambda")
        ax1.set_ylabel("relative proliferation")

        ax2.set_xscale('log')
        ax2.set_xlabel("lambda")
        ax2.set_ylabel("dosage (nm)")
        ax2.legend()

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()


A2058
C32
